In [9]:
import ast

source = "a = -5"
tree = ast.parse(source)

In [10]:
type(tree.body[0].value.operand.value)

int

In [17]:
source = 'int(\"15.10\")'
tree = ast.parse('int(\'15.10\')')

In [20]:
ast.get_source_segment(source, tree.body[0].value.args[0])[1:-1]

'15.10'

In [22]:
bool('False')

True

In [24]:
from tqdm import tqdm

for i, row in enumerate(tqdm([1,2,3])):
    print(row)

100%|██████████| 3/3 [00:00<00:00, 54471.48it/s]

1
2
3
